In [80]:
import pandas as pd
import math

In [81]:
df = pd.read_excel('data/nyc2010census_tabulation_equiv.xlsx', 
                   skiprows=4, dtype=str,
                  names=['borough', 'fips', 'borough_code', 'tract', 'puma', 'nta_code', 'nta_name'])
demo = pd.read_csv('data/demo.csv', index_col=False)
city_level = demo[demo['GEO_ID'].str.startswith('16')].to_dict('records')[0] # --> entire new york city estimates and margin of errors

In [82]:
df['GEO_ID'] = '1400000US36' + df['fips'] + df['tract']

In [83]:
df.head()

,borough,fips,borough_code,tract,puma,nta_code,nta_name,GEO_ID
0,Bronx,005,2,031000,3704,BX31,Allerton-Pelham Gardens,1400000US36005031000
1,Bronx,005,2,031200,3704,BX31,Allerton-Pelham Gardens,1400000US36005031200
2,Bronx,005,2,031400,3704,BX31,Allerton-Pelham Gardens,1400000US36005031400
3,Bronx,005,2,031600,3704,BX31,Allerton-Pelham Gardens,1400000US36005031600
4,Bronx,005,2,031800,3704,BX31,Allerton-Pelham Gardens,1400000US36005031800


In [84]:
demo.head()

,B01001_001E,B01001_001M,B01001_002E,B01001_002M,B01001_003E,B01001_003M,B01001_004E,B01001_004M,B01001_005E,B01001_005M,...,B03001_027E,B03001_027M,B03001_028E,B03001_028M,B03001_029E,B03001_029M,B03001_030E,B03001_030M,B03001_031E,B03001_031M
0,31611,-555555555,15700,61,711,25,843,84,777,85,...,68,33,31,23,13,14,0,24,24,20
1,1455846,-555555555,685636,195,55036,142,51974,1319,49368,1298,...,18154,1629,1698,383,1041,421,111,120,15304,1395
2,467669,-555555555,225685,101,13830,72,14245,516,14537,517,...,1833,321,649,223,412,229,0,27,772,209
3,53955,-555555555,26899,112,1474,59,1649,153,1626,168,...,172,111,24,24,39,33,0,27,109,104
4,81224,-555555555,41648,104,2056,79,2112,200,2062,196,...,287,83,71,57,4,6,0,27,212,65


In [85]:
df_demo = pd.merge(df[['nta_code', 'GEO_ID']], demo, how='left', left_on='GEO_ID', right_on='GEO_ID')

In [86]:
df_demo.head()

,nta_code,GEO_ID,B01001_001E,B01001_001M,B01001_002E,B01001_002M,B01001_003E,B01001_003M,B01001_004E,B01001_004M,...,B03001_027E,B03001_027M,B03001_028E,B03001_028M,B03001_029E,B03001_029M,B03001_030E,B03001_030M,B03001_031E,B03001_031M
0,BX31,1400000US36005031000,4806.0,408.0,2174.0,218.0,108.0,73.0,184.0,83.0,...,58.0,47.0,27.0,38.0,10.0,18.0,0.0,11.0,21.0,24.0
1,BX31,1400000US36005031200,1779.0,215.0,774.0,86.0,15.0,16.0,36.0,27.0,...,17.0,26.0,0.0,11.0,0.0,11.0,0.0,11.0,17.0,26.0
2,BX31,1400000US36005031400,2228.0,229.0,1106.0,142.0,50.0,25.0,10.0,15.0,...,11.0,11.0,11.0,11.0,0.0,11.0,0.0,11.0,0.0,11.0
3,BX31,1400000US36005031600,1899.0,217.0,999.0,149.0,20.0,19.0,61.0,42.0,...,24.0,38.0,0.0,11.0,0.0,11.0,0.0,11.0,24.0,38.0
4,BX31,1400000US36005031800,2001.0,318.0,963.0,186.0,63.0,48.0,26.0,23.0,...,13.0,20.0,0.0,11.0,0.0,11.0,0.0,11.0,13.0,20.0


### calculate estimate, moe, cv and percent estimate, percent moe

In [87]:
variables = list(demo.columns)
variables.remove('GEO_ID')
variables.remove('NAME')
var = list(set([i[:-1] for i in variables if i[:-1][-1] != 'P']))

In [88]:
var[:5]

['DP05_0078', 'B03001_004', 'B02015_014', 'B03001_019', 'B03001_021']

In [89]:
results = []
for i in df_demo.nta_code.unique():
    dff = df_demo[df_demo.nta_code == i]
    record = {}
    record['nta'] = i
    for v in var:
        e = dff[f'{v}E'].sum()
        m = math.sqrt(dff[f'{v}M'].apply(lambda x: x**2).sum())
        record[f'{v}E'] = e
        record[f'{v}M'] = m
    results.append(record)

In [90]:
r = pd.DataFrame(results)
r['GEO_ID'] = r.nta
r = r.rename(columns={'nta':'NAME'})

In [91]:
output = pd.concat([r, demo], sort=True)

In [92]:
output.head()

,B01001_001E,B01001_001M,B01001_002E,B01001_002M,B01001_003E,B01001_003M,B01001_004E,B01001_004M,B01001_005E,B01001_005M,...,DP05_0088E,DP05_0088M,DP05_0088PE,DP05_0088PM,DP05_0089E,DP05_0089M,DP05_0089PE,DP05_0089PM,GEO_ID,NAME
0,32127.0,1246.122386,14961.0,712.721545,694.0,178.555314,937.0,224.926655,1020.0,250.339769,...,9935.0,578.784934,NaN,NaN,11968.0,652.078983,NaN,NaN,BX31,BX31
1,57718.0,1775.077463,27570.0,1175.599422,2768.0,516.036820,1932.0,331.407604,1993.0,323.620148,...,13717.0,916.604604,NaN,NaN,16104.0,830.780958,NaN,NaN,BX05,BX05
2,29990.0,1432.393452,13655.0,964.579701,869.0,237.491052,1232.0,321.619340,1003.0,315.680218,...,7037.0,596.124148,NaN,NaN,8776.0,631.763405,NaN,NaN,BX06,BX06
3,38083.0,1482.243570,18710.0,1032.483898,1660.0,380.988189,1240.0,297.465964,1169.0,288.263768,...,10555.0,741.169346,NaN,NaN,12806.0,706.343401,NaN,NaN,BX07,BX07
4,35856.0,1580.290163,15986.0,1231.871341,1703.0,411.582313,1559.0,310.049996,1327.0,271.239746,...,8390.0,1023.547263,NaN,NaN,11047.0,687.457635,NaN,NaN,BX01,BX01


In [93]:
output.to_csv('data/demo_intermediate.csv', index=False)